In [6]:
import json
import pandas as pd

with open("/home/msaad/workspace/honors-thesis/data-collection/data/categorized_data.json", "r") as f:
    data = json.load(f)

# Load in the JSON file

In [16]:
list(data['academics'].keys())

['https://www2.brockport.edu/academics',
 'academic-affairs',
 'https://www2.brockport.edu/academics/academic_affairs/innovations_task_force/index.html',
 'https://www2.brockport.edu/academics/academic_affairs/master_plan',
 'https://www2.brockport.edu/academics/academic_affairs/meeting_minutes.html',
 'https://www2.brockport.edu/academics/accounting/finance_major.html',
 'advisement',
 'african-american',
 'https://www2.brockport.edu/academics/aging',
 'https://www2.brockport.edu/academics/aging/advanced-cert',
 'https://www2.brockport.edu/academics/aging/directory',
 'https://www2.brockport.edu/academics/american_democracy_project/constitution',
 'anthropology',
 'art',
 'biology',
 'https://www2.brockport.edu/academics/business-administration',
 'https://www2.brockport.edu/academics/calendar',
 'https://www2.brockport.edu/academics/calendar/archive',
 'career',
 'celt',
 'certification',
 'chemistry',
 'commencement',
 'communication',
 'https://www2.brockport.edu/academics/communit

In [122]:
df = pd.read_csv("data/full_cleaned_data.csv")

In [33]:
df = df.data.str.len().sort_values().index()

TypeError: 'Index' object is not callable

In [35]:
df.index = df['data'].str.len()
df = df.sort_index(ascending=False).reset_index(drop=True)

In [104]:
len(df.iloc[2590,1])

278

In [105]:
df

,url,data
0,https://www2.brockport.edu/academics/social-wo...,Welcome and Introduction\nWelcome to the Maste...
1,https://www2.brockport.edu/academics/nursing/h...,SUNY Brockport\nDepartment of Nursing\nReview ...
2,https://www2.brockport.edu/support/policies/ca...,-\nAcademic Affairs\n- Academic Dishonesty Pol...
3,https://www2.brockport.edu/academics/political...,"<picture class=""lw_image""> <source type=""image..."
4,https://www2.brockport.edu/academics/kinesiolo...,Policies and Procedures Manual (2022-2023)\nI....
...,...,...
2659,https://www2.brockport.edu/life/student-conduc...,Laura BronsinkDirector of Student Conduct (585...
2660,https://www2.brockport.edu/go/ems,There was a problem attempting to complete the...
2661,https://www2.brockport.edu/library/lookup,JavaScript must be enabled to use the system
2662,https://www2.brockport.edu/about/diversity/bia...,Sign in to your account


In [109]:
df[df.data.str.len() > 275]

,url,data
0,https://www2.brockport.edu,Main Page Content\nBe ThereEvents Calendar\nWh...
1,https://www2.brockport.edu/about,"About\nSUNY Brockport is a selective, national..."
2,https://www2.brockport.edu/about/accountability,Accountability & SUNY Brockport\nSUNY Brockpor...
3,https://www2.brockport.edu/about/accountabilit...,Student Learning Outcomes (SLOs)\nMain Page Co...
4,https://www2.brockport.edu/about/accountabilit...,University-wide Committees Joint Planning and ...
...,...,...
2659,https://www2.brockport.edu/support/university-...,V. Violence Against Women Act (VAWA) Crimes\nV...
2660,https://www2.brockport.edu/support/university-...,"Although we enjoy a safe campus, no area is to..."
2661,https://www2.brockport.edu/support/university-...,Sexual Assault and the Law\nSUNY Brockport has...
2662,https://www2.brockport.edu/support/university-...,The Student Patrol program allows students wor...


In [110]:
df[df['data'].str.len() > 275]

,url,data
0,https://www2.brockport.edu,Main Page Content\nBe ThereEvents Calendar\nWh...
1,https://www2.brockport.edu/about,"About\nSUNY Brockport is a selective, national..."
2,https://www2.brockport.edu/about/accountability,Accountability & SUNY Brockport\nSUNY Brockpor...
3,https://www2.brockport.edu/about/accountabilit...,Student Learning Outcomes (SLOs)\nMain Page Co...
4,https://www2.brockport.edu/about/accountabilit...,University-wide Committees Joint Planning and ...
...,...,...
2659,https://www2.brockport.edu/support/university-...,V. Violence Against Women Act (VAWA) Crimes\nV...
2660,https://www2.brockport.edu/support/university-...,"Although we enjoy a safe campus, no area is to..."
2661,https://www2.brockport.edu/support/university-...,Sexual Assault and the Law\nSUNY Brockport has...
2662,https://www2.brockport.edu/support/university-...,The Student Patrol program allows students wor...


In [127]:
df = pd.read_csv("data/full_cleaned_data.csv")

In [128]:
df['url'].str.contains("_").value_counts()

url
False    2379
True      285
Name: count, dtype: int64

In [129]:
df['url'] = df['url'].str.replace("_", "-")

In [130]:
df['url'].str.contains("_").value_counts()

url
False    2664
Name: count, dtype: int64

In [136]:
df.drop_duplicates(subset=["url"])

,url,data
0,https://www2.brockport.edu,Main Page Content\nBe ThereEvents Calendar\nWh...
1,https://www2.brockport.edu/about,"About\nSUNY Brockport is a selective, national..."
2,https://www2.brockport.edu/about/accountability,Accountability & SUNY Brockport\nSUNY Brockpor...
3,https://www2.brockport.edu/about/accountabilit...,Student Learning Outcomes (SLOs)\nMain Page Co...
4,https://www2.brockport.edu/about/accountabilit...,University-wide Committees Joint Planning and ...
...,...,...
2659,https://www2.brockport.edu/support/university-...,V. Violence Against Women Act (VAWA) Crimes\nV...
2660,https://www2.brockport.edu/support/university-...,"Although we enjoy a safe campus, no area is to..."
2661,https://www2.brockport.edu/support/university-...,Sexual Assault and the Law\nSUNY Brockport has...
2662,https://www2.brockport.edu/support/university-...,The Student Patrol program allows students wor...


In [137]:
len(df)

2664

In [140]:
df['url'].str.endswith(".html").value_counts()

url
False    2396
True      268
Name: count, dtype: int64

In [143]:
# remove the .html and deduplicate, just like _
print(f"Before {len(df)}")

df['url'] = df['url'].str.removesuffix(".html")

df = df.drop_duplicates(subset=['url'])

print(f"After {len(df)}")

Before 2664
After 2657


In [141]:
"test.html".removesuffix(".html")

'test'